#### Move to your working directory

In [31]:
import os
os.chdir(os.getcwd()+"/pratical-llms")

### WHat is HQQ?
HQQ is a fast and accurate model quantizer that skips the need for calibration data. It's super simple to implement (just a few lines of code for the optimizer). It can crunch through quantizing the Llama2-70B model in only 4 minutes!

![hqq_table_comparison.png](images/hqq_table.png)

In [2]:
!git clone https://github.com/mobiusml/hqq.git

fatal: destination path 'hqq' already exists and is not an empty directory.


⚡️ Tip	Check organization access: https://github.com/settings/connections/applications/c7457225b242a94d60c6



In [2]:
!pip install hqq/.

Processing ./hqq
  Preparing metadata (setup.py) ... done
  Using cached timm-0.9.16-py3-none-any.whl.metadata (38 kB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl.metadata (1.8 kB)
Using cached timm-0.9.16-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 28.0 MB/s eta 0:00:0000:0100:01
  Created wheel for hqq: filename=hqq-0.1.6.post2-py3-none-any.whl size=46058 sha256=ce077cf8c21899a3505ff43b74696c0de3abfb4c924db34a55bb5fb401b912dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-7sql2ht6/wheels/8a/a6/e0/996e7914924693544ca3e4d355114e62dde67a58d263ea43cb
Successfully built hqq
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia

In [11]:
!pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.5.7.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.5.7-cp310-cp310-linux_x86_64.whl size=120853563 sha256=bbe6f77fd0899f8a125a5bdcf734b660c4c88e81c9b51c7ce98ebeba44dc6fa0
  Stored in directory: /home/zeus/.cache/pip/wheels/13/96/ed/bcac89c56b606421f99b45b16a94db5d0f2b6b4eaf8bac4d01
Successfully built flash-attn


In [3]:
!num_threads=8; OMP_NUM_THREADS=$num_threads CUDA_VISIBLE_DEVICES=0

In [4]:
BITS = 4
GROUP_SIZE = 128

In [5]:
import torch
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *

In [6]:
quant_config = BaseQuantizeConfig(
    nbits=BITS,
    group_size=GROUP_SIZE
)

In [8]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [12]:
model = HQQModelForCausalLM.from_pretrained(
    model_id,
    cache_dir=".",
    attn_implementation="flash_attention_2"
)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_dev

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.quantize_model(quant_config=quant_config, device='cuda')

100%|██████████| 22/22 [00:12<00:00,  1.71it/s]


In [14]:
# Save model and tokenizer
save_folder = model_id + "-HQQ"
model.save_quantized(save_folder)
tokenizer.save_pretrained(save_folder)

('TinyLlama/TinyLlama-1.1B-Chat-v1.0-HQQ/tokenizer_config.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-HQQ/special_tokens_map.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-HQQ/tokenizer.model',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-HQQ/added_tokens.json',
 'TinyLlama/TinyLlama-1.1B-Chat-v1.0-HQQ/tokenizer.json')

That's it. 

## Inference

In [32]:
import time

def time_it(start,end):
    nano = end-start
    return nano/1e9

In [21]:
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer

In [33]:
save_dir_or_hfhub = "TinyLlama/TinyLlama-1.1B-Chat-v1.0-HQQ"
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
device ="cuda"
max_token = 100

In [18]:
model = HQQModelForCausalLM.from_quantized(save_dir_or_hfhub, device=device)

100%|██████████| 22/22 [00:00<00:00, 1101.25it/s]


In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_id) 

In [34]:
text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to(device)
start = time.time_ns()
outputs = model.generate(**inputs, max_new_tokens=max_token)
end = time.time_ns()
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
t = time_it(start,end)
print("Seconds:",t)
print("Token/s",len(outputs[0])/t)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Hello my name is John Smith and I am a great fan of the movie "The Godfather". I have been a fan of this movie since I was a child and I have seen it many times. I have also read the book and I have read the movie. I have also written a review of the movie and I have written a review of the book. I have also written a review of the movie and I have written a review of the book. I have also written a review of the movie and I have
Seconds: 8.930624513
Token/s 11.757296463103465


In [35]:
print(f"Model size: {model.get_memory_footprint():,} bytes")

Model size: 758,306,176 bytes
